In [12]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import os
import itertools
from sklearn.model_selection import KFold

In [13]:
prefix = os.path.abspath(os.path.join(os.getcwd(), ".."))
text_features = np.load(os.path.join(prefix, 'Features/TextWhole/mod_samples_clf_avg.npz'))['arr_0']
text_targets = np.load(os.path.join(prefix, 'Features/TextWhole/mod_labels_clf_avg.npz'))['arr_0']
text_dep_idxs_tmp = np.where(text_targets == 1)[0]
text_non_idxs = np.where(text_targets == 0)[0]

In [14]:
print(text_features.shape)
print(text_targets.shape)
print(np.unique(text_targets))

(162, 1, 768)
(162,)
[0 1]


In [15]:
## Run this if the random indices do not give trainable model for fusenet.

# prefix = os.path.abspath(os.path.join(os.getcwd(), ".."))
# text_features = np.load(os.path.join(prefix, 'Features/TextWhole/mod_samples_clf_avg.npz'))['arr_0']
# text_targets = np.load(os.path.join(prefix, 'Features/TextWhole/mod_labels_clf_avg.npz'))['arr_0']
# text_dep_idxs_tmp = np.where(text_targets == 1)[0]
# text_non_idxs = np.where(text_targets == 0)[0]

# kf = KFold(n_splits=3, shuffle=True)
# fold = 1
# for train_idxs_tmp, test_idxs_tmp in kf.split(text_features):
#     ## Code to save the .npy files.
#     np.save(os.path.join(prefix, 'Features/TextWhole/mod_train_idxs_{}.npy'.format(fold)), train_idxs_tmp)
#     np.save(os.path.join(prefix, 'Features/TextWhole/mod_test_idxs_{}.npy'.format(fold)), test_idxs_tmp)
#     fold += 1

config = {
    'num_classes': 2,
    'dropout': 0.5,
    'rnn_layers': 2,
    'embedding_size': 768,
    'batch_size': 4,
    'epochs': 150,
    'learning_rate': 1e-5,
    'hidden_dims': 128,
    'bidirectional': True,
    'cuda': False,
}

In [16]:
class TextBiLSTM(nn.Module):
    def __init__(self, config):
        super(TextBiLSTM, self).__init__()
        self.num_classes = config['num_classes']
        self.learning_rate = config['learning_rate']
        self.dropout = config['dropout']
        self.hidden_dims = config['hidden_dims']
        self.rnn_layers = config['rnn_layers']
        self.embedding_size = config['embedding_size']
        self.bidirectional = config['bidirectional']

        self.build_model()
        self.init_weight()
        
    def init_weight(net):
        for name, param in net.named_parameters():
            if 'ln' not in name:
                if 'bias' in name:
                    nn.init.constant_(param, 0.0)
                elif 'weight' in name:
                    nn.init.xavier_uniform_(param)

    def build_model(self):
        self.attention_layer = nn.Sequential(
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(inplace=True)
        )
        self.lstm_net = nn.LSTM(self.embedding_size, self.hidden_dims,
                                num_layers=self.rnn_layers, dropout=self.dropout,
                                bidirectional=self.bidirectional)
        self.fc_out = nn.Sequential(
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.num_classes),
            nn.Softmax(dim=1),
        )
        self.ln1 = nn.LayerNorm(self.embedding_size)
        self.ln2 = nn.LayerNorm(self.hidden_dims)

    def attention_net_with_w(self, lstm_out, lstm_hidden):
        '''
        :param lstm_out:    [batch_size, len_seq, n_hidden * 2]
        :param lstm_hidden: [batch_size, num_layers * num_directions, n_hidden]
        :return: [batch_size, n_hidden]
        '''
        lstm_tmp_out = torch.chunk(lstm_out, 2, -1)
        # h [batch_size, time_step, hidden_dims]
        h = lstm_tmp_out[0] + lstm_tmp_out[1]
        # h = lstm_out
        # [batch_size, num_layers * num_directions, n_hidden]
        lstm_hidden = torch.sum(lstm_hidden, dim=1)
        # [batch_size, 1, n_hidden]
        lstm_hidden = lstm_hidden.unsqueeze(1)
        # atten_w [batch_size, 1, hidden_dims]
        atten_w = self.attention_layer(lstm_hidden)
        # m [batch_size, time_step, hidden_dims]
        m = nn.Tanh()(h)
        # atten_context [batch_size, 1, time_step]
        atten_context = torch.bmm(atten_w, m.transpose(1, 2))
        # softmax_w [batch_size, 1, time_step]
        softmax_w = F.softmax(atten_context, dim=-1)
        # context [batch_size, 1, hidden_dims]
        context = torch.bmm(softmax_w, h)
        result = context.squeeze(1)
        return result

    def forward(self, x):
        # x : [len_seq, batch_size, embedding_dim]
        x = x.permute(1, 0, 2)
        # x = self.ln1(x)
        output, (final_hidden_state, _) = self.lstm_net(x)
        # output : [batch_size, len_seq, n_hidden * 2]
        output = output.permute(1, 0, 2)
        # final_hidden_state : [batch_size, num_layers * num_directions, n_hidden]
        final_hidden_state = final_hidden_state.permute(1, 0, 2)
        # final_hidden_state = torch.mean(final_hidden_state, dim=0, keepdim=True)
        # atten_out = self.attention_net(output, final_hidden_state)
        atten_out = self.attention_net_with_w(output, final_hidden_state)
        # atten_out = self.ln2(atten_out)
        return self.fc_out(atten_out)

In [17]:
def save(model, filename):
    save_filename = '{}.pt'.format(filename)
    torch.save(model, save_filename)
    print('Saved as %s' % save_filename)

def standard_confusion_matrix(y_test, y_test_pred):
    """
    Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    Parameters
    ----------
    y_true : ndarray - 1D
    y_pred : ndarray - 1D

    Returns
    -------
    ndarray - 2D
    """
    [[tn, fp], [fn, tp]] = confusion_matrix(y_test, y_test_pred)
    return np.array([[tp, fp], [fn, tn]])

def model_performance(y_test, y_test_pred_proba):
    """
    Evaluation metrics for network performance.
    """
    y_test_pred = y_test_pred_proba.data.max(1, keepdim=True)[1]

    # Computing confusion matrix for test dataset
    conf_matrix = standard_confusion_matrix(y_test, y_test_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    return y_test_pred, conf_matrix

In [18]:
def train(epoch, train_idxs):
    global lr, train_acc
    model.train()
    batch_idx = 1
    total_loss = 0
    correct = 0
    X_train = text_features[train_idxs]
    Y_train = text_targets[train_idxs]
    for i in range(0, X_train.shape[0], config['batch_size']):
        if i + config['batch_size'] > X_train.shape[0]:
            x, y = X_train[i:], Y_train[i:]
        else:
            x, y = X_train[i:(i + config['batch_size'])], Y_train[i:(
                i + config['batch_size'])]
       
        if config['cuda']:
            x, y = Variable(torch.from_numpy(x).type(torch.FloatTensor), requires_grad=True).cuda(), Variable(torch.from_numpy(y)).cuda()
        else:
            x, y = Variable(torch.from_numpy(x).type(torch.FloatTensor), requires_grad=True), \
                Variable(torch.from_numpy(y))

        optimizer.zero_grad()
        output = model(x)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).cpu().sum()
        loss = criterion(output, y.long())
        loss.backward()
        optimizer.step()
        batch_idx += 1
        total_loss += loss.item()

    train_acc = correct
    print('Train Epoch: {:2d}\t Learning rate: {:.4f}\tLoss: {:.6f}\t Accuracy: {}/{} ({:.0f}%)\n '.format(epoch + 1, config['learning_rate'], total_loss, correct, X_train.shape[0], 100. * correct / X_train.shape[0]))

In [19]:

def evaluate(model, test_idxs, fold, train_idxs):
    model.eval()
    batch_idx = 1
    total_loss = 0
    global max_f1, max_acc, min_mae, X_test_lens, max_prec, max_rec
    pred = np.array([])
    with torch.no_grad():
        if config['cuda']:
            x, y = Variable(torch.from_numpy(text_features[test_idxs]).type(torch.FloatTensor), requires_grad=True).cuda(),\
                Variable(torch.from_numpy(text_targets[test_idxs])).cuda()
        else:
            x, y = Variable(torch.from_numpy(text_features[test_idxs]).type(torch.FloatTensor), requires_grad=True), \
                Variable(torch.from_numpy(text_targets[test_idxs])).type(torch.LongTensor)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y.long())
        total_loss += loss.item()
        y_test_pred, conf_matrix = model_performance(y, output.cpu())
        
        accuracy = float(conf_matrix[0][0] + conf_matrix[1][1]) / np.sum(conf_matrix)
        if conf_matrix[0][0] + conf_matrix[0][1] != 0:
            precision = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[0][1])
        else:
            precision = 0
        recall = float(conf_matrix[0][0]) / (conf_matrix[0][0] + conf_matrix[1][0])
        f1_score = 2 * (precision * recall) / (precision + recall)
        
        print("Accuracy: {}".format(accuracy))
        print("Precision: {}".format(precision))
        print("Recall: {}".format(recall))
        print("F1-Score: {}\n".format(f1_score))
        print('=' * 89)

        if max_f1 <= f1_score and train_acc > len(train_idxs)*0.9 and f1_score > 0.5:
            max_f1 = f1_score
            max_acc = accuracy
            max_rec = recall
            max_prec = precision
            
            save(model, os.path.join(prefix, 'Model/ClassificationWhole/Text/BiLSTM_{}_{:.2f}_{}'.format(config['hidden_dims'], max_f1, fold)))
            
            print('*' * 64)
            print('model saved: f1: {}\tacc: {}'.format(max_f1, max_acc))
            print('*' * 64)

    return total_loss

In [20]:
def get_param_group(model):
    nd_list = []
    param_list = []
    for name, param in model.named_parameters():
        if 'ln' in name:
            nd_list.append(param)
        else:
            param_list.append(param)
    return [{'params': param_list, 'weight_decay': 1e-5}, {'params': nd_list, 'weight_decay': 0}]

In [21]:
train_idxs_tmps = [np.load(os.path.join(prefix, 'Features/TextWhole/mod_train_idxs_1.npy'), allow_pickle=True),
                   np.load(os.path.join(prefix, 'Features/TextWhole/mod_train_idxs_2.npy'), allow_pickle=True),
                   np.load(os.path.join(prefix, 'Features/TextWhole/mod_train_idxs_3.npy'), allow_pickle=True)]
fold = 1

In [22]:
for idx_idx, train_idxs_tmp in enumerate(train_idxs_tmps):
    test_idxs_tmp = list(set(list(text_dep_idxs_tmp) + list(text_non_idxs)) - set(train_idxs_tmp))
    train_idxs, test_idxs = [], []

    for idx in train_idxs_tmp:
        if idx in text_dep_idxs_tmp:
            feat = text_features[idx]
            count = 0
            resample_idxs = [0, 1, 2, 3, 4, 5]
            for i in itertools.permutations(feat, feat.shape[0]):
                if count in resample_idxs:
                    text_features = np.vstack((text_features, np.expand_dims(list(i), 0)))
                    text_targets = np.hstack((text_targets, 1))
                    train_idxs.append(len(text_features) - 1)
                count += 1
        else:
            train_idxs.append(idx)

    for idx in test_idxs_tmp:
        if idx in text_dep_idxs_tmp:
            feat = text_features[idx]
            count = 0
            resample_idxs = [0, 1, 4, 5]
            for i in itertools.permutations(feat, feat.shape[0]):
                if count in resample_idxs:
                    text_features = np.vstack((text_features, np.expand_dims(list(i), 0)))
                    text_targets = np.hstack((text_targets, 1))
                    test_idxs.append(len(text_features) - 1)
                count += 1
        else:
            test_idxs.append(idx)

    model = TextBiLSTM(config)

    param_group = get_param_group(model)
    optimizer = optim.Adam(param_group , lr=config['learning_rate'])
    criterion = nn.CrossEntropyLoss()
    max_f1 = -1
    max_acc = -1
    max_rec = -1
    max_prec = -1
    train_acc = -1
    
    for epoch in range(config['epochs']):
        train(epoch, train_idxs)
        total_loss = evaluate(model, test_idxs, fold, train_idxs)
    fold += 1


Train Epoch:  1	 Learning rate: 0.0000	Loss: 18.680583	 Accuracy: 53/108 (49%)
 
Confusion Matrix:
[[ 0  0]
 [11 43]]
Accuracy: 0.7962962962962963
Precision: 0
Recall: 0.0
F1-Score: nan



C:\Users\meena\AppData\Local\Temp\ipykernel_7124\1959198720.py:27: RuntimeWarning: invalid value encountered in scalar divide
  f1_score = 2 * (precision * recall) / (precision + recall)


Train Epoch:  2	 Learning rate: 0.0000	Loss: 18.485469	 Accuracy: 68/108 (63%)
 
Confusion Matrix:
[[ 0  0]
 [11 43]]
Accuracy: 0.7962962962962963
Precision: 0
Recall: 0.0
F1-Score: nan



C:\Users\meena\AppData\Local\Temp\ipykernel_7124\1959198720.py:27: RuntimeWarning: invalid value encountered in scalar divide
  f1_score = 2 * (precision * recall) / (precision + recall)


Train Epoch:  3	 Learning rate: 0.0000	Loss: 18.288588	 Accuracy: 77/108 (71%)
 
Confusion Matrix:
[[ 0  0]
 [11 43]]
Accuracy: 0.7962962962962963
Precision: 0
Recall: 0.0
F1-Score: nan



...

C:\Users\meena\AppData\Local\Temp\ipykernel_7124\1959198720.py:27: RuntimeWarning: invalid value encountered in scalar divide
  f1_score = 2 * (precision * recall) / (precision + recall)


Train Epoch: 149	 Learning rate: 0.0000	Loss: 13.797548	 Accuracy: 87/108 (81%)
 
Confusion Matrix:
[[ 0  0]
 [ 7 47]]
Accuracy: 0.8703703703703703
Precision: 0
Recall: 0.0
F1-Score: nan



C:\Users\meena\AppData\Local\Temp\ipykernel_7124\1959198720.py:27: RuntimeWarning: invalid value encountered in scalar divide
  f1_score = 2 * (precision * recall) / (precision + recall)


Train Epoch: 150	 Learning rate: 0.0000	Loss: 13.890705	 Accuracy: 87/108 (81%)
 
Confusion Matrix:
[[ 0  0]
 [ 7 47]]
Accuracy: 0.8703703703703703
Precision: 0
Recall: 0.0
F1-Score: nan



C:\Users\meena\AppData\Local\Temp\ipykernel_7124\1959198720.py:27: RuntimeWarning: invalid value encountered in scalar divide
  f1_score = 2 * (precision * recall) / (precision + recall)
